In [34]:
%%pyspark
df = spark.read.load('abfss://housepricing@datalakezain.dfs.core.windows.net/input/housing_in_london_monthly_variables.csv', format='csv'
## If header exists uncomment line below
, header=True
)

StatementMeta(sparkpool, 14, 35, Finished, Available)

In [67]:
from pyspark.sql.functions import col,expr,count,sum,when

#changing col data type by using expr
new_df=df.select(expr('cast(date as date)'),expr('area as area'),expr('cast(average_price as int) as ap'),df.code,
expr('cast(houses_sold as int)'),expr('cast(borough_flag as boolean)'))

#changing col data type by using selectexpr
#new_df=df.selectExpr('cast(date as date)','area as a','cast(average_price as int) as ap','code',
#'cast(houses_sold as int)','cast(borough_flag as boolean)')

StatementMeta(sparkpool, 14, 68, Finished, Available)

In [68]:
# dropping cols
drop_col=new_df.drop("code")

StatementMeta(sparkpool, 14, 69, Finished, Available)

In [69]:
# group by with where and having clause 
agg=new_df.select('ap','houses_sold').where("houses_sold<10").groupBy('ap').agg(sum('houses_sold').alias('houses')).filter('houses < 5')

StatementMeta(sparkpool, 14, 70, Finished, Available)

In [71]:
# adding new column based on values of current column
a=new_df.withColumn('is_active',when(col('borough_flag')==1,'yes').when(col('borough_flag')==0,'no').otherwise('No'))

StatementMeta(sparkpool, 14, 72, Finished, Available)

In [91]:
from pyspark.sql.functions import regexp_replace,isnull,isnan

# changing values with a column using regex
regex_col=a.withColumn('City',regexp_replace(col('area'),"city of l","L"))

StatementMeta(sparkpool, 14, 92, Finished, Available)

In [107]:
df.filter(~ df.no_of_crimes.isNull()).show()

StatementMeta(sparkpool, 14, 108, Finished, Available)

+----------+--------------+-------------+---------+-----------+------------+------------+
|      date|          area|average_price|     code|houses_sold|no_of_crimes|borough_flag|
+----------+--------------+-------------+---------+-----------+------------+------------+
|2001-01-01|city of london|       284262|E09000001|         24|         0.0|           1|
|2001-02-01|city of london|       198137|E09000001|         37|         0.0|           1|
|2001-03-01|city of london|       189033|E09000001|         44|         0.0|           1|
|2001-04-01|city of london|       205494|E09000001|         38|         0.0|           1|
|2001-05-01|city of london|       223459|E09000001|         30|         0.0|           1|
|2001-06-01|city of london|       242435|E09000001|         36|         0.0|           1|
|2001-07-01|city of london|       230700|E09000001|         40|         0.0|           1|
|2001-08-01|city of london|       231632|E09000001|         30|         0.0|           1|
|2001-09-0

In [114]:
from pyspark.sql.functions import row_number,desc
from pyspark.sql.window import Window

# checking most number of houses sold by each code using window

w=Window.partitionBy(col('code')).orderBy(col('houses_sold').desc())
rn=row_number().over(w)

# selecting 2nd highest number of houses sold
a=df.withColumn('rn',rn).where('rn==2')



StatementMeta(sparkpool, 14, 115, Finished, Available)

In [115]:
a.show()

StatementMeta(sparkpool, 14, 116, Finished, Available)

+----------+--------------------+-------------+---------+-----------+------------+------------+---+
|      date|                area|average_price|     code|houses_sold|no_of_crimes|borough_flag| rn|
+----------+--------------------+-------------+---------+-----------+------------+------------+---+
|1999-12-01|      city of london|       169309|E09000001|         97|        null|           1|  2|
|2013-03-01|barking and dagenham|       169414|E09000002|         96|      1449.0|           1|  2|
|1999-07-01|              barnet|       136102|E09000003|        865|        null|           1|  2|
|2002-05-01|              bexley|       131293|E09000004|        633|      1694.0|           1|  2|
|2009-05-01|               brent|       261460|E09000005|         92|      2387.0|           1|  2|
|2006-08-01|             bromley|       253406|E09000006|        856|      2144.0|           1|  2|
|2009-02-01|              camden|       453141|E09000007|         89|      2458.0|           1|  2|
